# Data Analysis
### This part is to do data analisis. Including win_rate, duration, high win_rate heroes. We use bokeh to visualize the data

In [1]:
# Run some setup code for this notebook.
from __future__ import print_function

import pandas as pd
import numpy as np
import scipy.special
import json

from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

In [2]:
#Read all data
df= pd.read_csv('out.csv')
df.shape

(114664, 33)

### The win rate of radiant and dire

In [3]:
# When df['radiant_win']==True, that means radiant win the game
radiant_win=len(df[df['radiant_win']==True])
print("The win rate of radiant is "+ str(radiant_win/len(df)))

The win rate of radiant is 0.5698126700620945


In [4]:
dire_win=len(df[df['radiant_win']==False])
print("The win rate of dire is "+ str(dire_win/len(df)))

The win rate of dire is 0.43018732993790554


In [5]:
from bokeh.io import show, output_file
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
output_notebook()
wins = ['Radiant_win:56.98%', 'Dire_win:43.02%']
counts = [62351, 52313]

source = ColumnDataSource(data=dict(wins=wins, counts=counts, color=['#d53e4f','green']))

p = figure(x_range=wins, y_range=(0,80000), plot_height=350, width=400, title="Wins Counts",
           toolbar_location=None, tools="")

p.vbar(x='wins', top='counts',width=0.5, color='color', legend='wins', source=source)

p.xgrid.grid_line_color = None
p.legend.orientation = "horizontal"
p.legend.location = "top_center"

show(p)

Loading BokehJS ...

### The distibution of time duration. Most game spend 20 minutes to 1 hour.

In [6]:
def make_plot(title, hist, edges):
    p = figure(title=title, tools='', background_fill_color="#fafafa")
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
           fill_color="navy", line_color="white", alpha=0.5,legend='duration')
   
    p.y_range.start = 0
    p.legend.location = "center_right"
    p.legend.background_fill_color = "#fefefe"
    p.xaxis.axis_label = 'Duration/s'
    p.yaxis.axis_label = 'Pr(Durarion)'
    p.grid.grid_line_color="white"
    return p

# Normal Distribution


measured = np.array(df['duration'])
hist, edges = np.histogram(measured, density=True, bins=50)

p1 = make_plot("Dutration Distribution", hist, edges)

show(p1)

### The distribution of winsocre. Winscore is the difference of killing numbers. Basically, high win_score means high win rate

In [7]:
def make_plot(title, hist, edges):
    p = figure(title=title, tools='', background_fill_color="#fafafa")
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
           fill_color="lightgreen", line_color="white", alpha=0.5,legend='Win_score')
   
    p.y_range.start = 0
    p.legend.location = "center_right"
    p.legend.background_fill_color = "#fefefe"
    p.xaxis.axis_label = 'Win_score/s'
    p.yaxis.axis_label = 'Pr(Win_score)'
    p.grid.grid_line_color="white"
    return p

# Normal Distribution


measured = np.array(df['win_score'])
hist, edges = np.histogram(measured, density=True, bins=50)

p1 = make_plot("Win_score Distribution", hist, edges)


show(p1)

### This part is to calculate the frequency and win rate of each hero

In [8]:
#heroescount is all game counts for each hero
#winheroescount is win game counts for each hero
#loseheroescount is lose game counts for each hero

heroescount=np.zeros(130)
winheroescount=np.zeros(130)
loseheroescount=np.zeros(130)

In [9]:
from progressbar import ProgressBar
pbar = ProgressBar()
for i in pbar(range(0,len(df))):
    if(df.iloc[i,0]==True):
        for j in range(23,28):
            heroescount[df.iloc[i,j]]=heroescount[df.iloc[i,j]]+1
            winheroescount[df.iloc[i,j]]=winheroescount[df.iloc[i,j]]+1
        for j in range(28,33):
            heroescount[df.iloc[i,j]]=heroescount[df.iloc[i,j]]+1
            loseheroescount[df.iloc[i,j]]=loseheroescount[df.iloc[i,j]]+1    
    if(df.iloc[i,0]==False):
        for j in range(23,28):
            heroescount[df.iloc[i,j]]=heroescount[df.iloc[i,j]]+1
            loseheroescount[df.iloc[i,j]]=loseheroescount[df.iloc[i,j]]+1
        for j in range(28,33) :
            heroescount[df.iloc[i,j]]=heroescount[df.iloc[i,j]]+1
            winheroescount[df.iloc[i,j]]=winheroescount[df.iloc[i,j]]+1

100% |########################################################################|


### Get the hero names from heroes.json. Translate the hero ID to hero name

In [10]:
#The name of heroes
heroesname=[]
for i in range(0,130):
    heroesname.append("empty")

In [11]:
fopen=open('heroes.json','r')
heroes=json.load(fopen)
fopen.close()

In [12]:
heroes[2]

{'id': 3,
 'name': 'npc_dota_hero_bane',
 'localized_name': 'Bane',
 'primary_attr': 'int',
 'attack_type': 'Ranged',
 'roles': ['Support', 'Disabler', 'Nuker', 'Durable'],
 'legs': 4}

In [13]:
for i in range(0,len(heroes)):
    heroesname[heroes[i].get('id')]=heroes[i].get('localized_name')

In [14]:
heroesname

['empty',
 'Anti-Mage',
 'Axe',
 'Bane',
 'Bloodseeker',
 'Crystal Maiden',
 'Drow Ranger',
 'Earthshaker',
 'Juggernaut',
 'Mirana',
 'Morphling',
 'Shadow Fiend',
 'Phantom Lancer',
 'Puck',
 'Pudge',
 'Razor',
 'Sand King',
 'Storm Spirit',
 'Sven',
 'Tiny',
 'Vengeful Spirit',
 'Windranger',
 'Zeus',
 'Kunkka',
 'empty',
 'Lina',
 'Lion',
 'Shadow Shaman',
 'Slardar',
 'Tidehunter',
 'Witch Doctor',
 'Lich',
 'Riki',
 'Enigma',
 'Tinker',
 'Sniper',
 'Necrophos',
 'Warlock',
 'Beastmaster',
 'Queen of Pain',
 'Venomancer',
 'Faceless Void',
 'Wraith King',
 'Death Prophet',
 'Phantom Assassin',
 'Pugna',
 'Templar Assassin',
 'Viper',
 'Luna',
 'Dragon Knight',
 'Dazzle',
 'Clockwerk',
 'Leshrac',
 "Nature's Prophet",
 'Lifestealer',
 'Dark Seer',
 'Clinkz',
 'Omniknight',
 'Enchantress',
 'Huskar',
 'Night Stalker',
 'Broodmother',
 'Bounty Hunter',
 'Weaver',
 'Jakiro',
 'Batrider',
 'Chen',
 'Spectre',
 'Ancient Apparition',
 'Doom',
 'Ursa',
 'Spirit Breaker',
 'Gyrocopter',
 '

In [15]:
heroesname

['empty',
 'Anti-Mage',
 'Axe',
 'Bane',
 'Bloodseeker',
 'Crystal Maiden',
 'Drow Ranger',
 'Earthshaker',
 'Juggernaut',
 'Mirana',
 'Morphling',
 'Shadow Fiend',
 'Phantom Lancer',
 'Puck',
 'Pudge',
 'Razor',
 'Sand King',
 'Storm Spirit',
 'Sven',
 'Tiny',
 'Vengeful Spirit',
 'Windranger',
 'Zeus',
 'Kunkka',
 'empty',
 'Lina',
 'Lion',
 'Shadow Shaman',
 'Slardar',
 'Tidehunter',
 'Witch Doctor',
 'Lich',
 'Riki',
 'Enigma',
 'Tinker',
 'Sniper',
 'Necrophos',
 'Warlock',
 'Beastmaster',
 'Queen of Pain',
 'Venomancer',
 'Faceless Void',
 'Wraith King',
 'Death Prophet',
 'Phantom Assassin',
 'Pugna',
 'Templar Assassin',
 'Viper',
 'Luna',
 'Dragon Knight',
 'Dazzle',
 'Clockwerk',
 'Leshrac',
 "Nature's Prophet",
 'Lifestealer',
 'Dark Seer',
 'Clinkz',
 'Omniknight',
 'Enchantress',
 'Huskar',
 'Night Stalker',
 'Broodmother',
 'Bounty Hunter',
 'Weaver',
 'Jakiro',
 'Batrider',
 'Chen',
 'Spectre',
 'Ancient Apparition',
 'Doom',
 'Ursa',
 'Spirit Breaker',
 'Gyrocopter',
 '

### Calculate the heroes frequency. Show top30 most common used heroes

In [16]:
# The frequency
heroesfrequency=np.argsort(-heroescount)

In [17]:
heroesfrequency

array([ 14,  26,  44,   8,  47,  74,  35, 104,  23,  62, 114,  27,  54,
        93,  21,   1,   7,  70,   2,  41,   5,  99,   6,  86,  84,  18,
        32,  53,  28,  95,  11,   4,  71,  36,   9,  50,  81,  34,  22,
        42, 120, 101,  30, 121,  63,  29, 100,  48,  25,  17,  59,  31,
       106,  75,  64,  12,  94,  73,  69, 105,  15,  67,  56,  40,  39,
        98,  19,  68,  20,  49,  10, 110,  97, 109,  45,  33,  85,  60,
        87,  88,  16,  96,  90,  51, 112,  83, 119,  37,  58, 108,  52,
        76,  13, 111,   3, 102,  72,  82,  46,  57,  55,  79, 107,  78,
        91,  43,  89,  77, 103,  80,  65,  38, 113,  61,  66,  92,   0,
       117, 118, 115, 128,  24, 122, 123, 124, 125, 126, 127, 116, 129],
      dtype=int64)

In [18]:
from bokeh.io import show, output_file
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.palettes import Spectral5
output_notebook()
top30heroes=[]
for j in range(0,30):
    top30heroes.append(heroesname[heroesfrequency[j]])
counts = heroescount[heroesfrequency[0:30]]

source = ColumnDataSource(data=dict(top30heroes=top30heroes, counts=counts ))

p = figure(x_range=top30heroes, y_range=(0,50000), plot_height=350, width=500, title="The Most Commonly Used Heroes",
           toolbar_location=None, tools="")

p.vbar(x='top30heroes', top='counts',width=0.7, color=Spectral5[3], source=source)

p.xgrid.grid_line_color = None
p.xaxis.major_label_orientation = 1.2
p.x_range.range_padding = 0.05

show(p)

Loading BokehJS ...

### The top30 win rate of heroes

In [19]:
heroeswinrate=winheroescount/heroescount
sortedhereoeswinrate=np.argsort(-heroeswinrate)
np.argsort(loseheroescount/heroescount)

C:\Users\wenyuyang\AppData\Local\Continuum\anaconda3\envs\DOTA\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.
C:\Users\wenyuyang\AppData\Local\Continuum\anaconda3\envs\DOTA\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


array([ 47,  70, 108,  54,  81,  50,   5,  36,  44,  29,  32,  37,  27,
        20,  42,  62,   4,  64,  85,  18,  89,  55,  67,  51,  33,  84,
        56,  95, 112, 102,  77,  68,  14,  75, 120,  22, 101,  26, 114,
        28,  60,  38,  78,  88,   8,  87,   0,   9,  57, 121,  23,  48,
        99,  92,  49,  73,  21,  69,  15,  30,   3,  83, 111,  53,  79,
         7,  52,  31, 104,  40,  71,   2,  74, 119,   6, 103,  96,  93,
        65, 110, 113,  41, 109,  72,   1,  90,  94,  63,  17,  16,  91,
       106,  45,  82, 105,  35,  58,  12,  66,  43,  59,  11,  86,  97,
        10,  39, 100, 107,  98,  19,  25,  34,  80,  13,  61,  46,  76,
       115, 116, 117, 118,  24, 122, 123, 124, 125, 126, 127, 128, 129],
      dtype=int64)

In [20]:
from bokeh.io import show, output_file
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.palettes import Spectral5
output_notebook()
top30heroes=[]
for j in range(0,30):
    top30heroes.append(heroesname[sortedhereoeswinrate[j]])
counts = heroeswinrate[sortedhereoeswinrate[0:30]]

source = ColumnDataSource(data=dict(top30heroes=top30heroes, counts=counts ))

p = figure(x_range=top30heroes, y_range=(0.4,0.6), plot_height=500, width=800, title="Heroes win rates",
           toolbar_location=None, tools="")

p.vbar(x='top30heroes', top='counts',width=0.7, color=Spectral5[4], source=source)

p.xgrid.grid_line_color = None
p.xaxis.major_label_orientation = 1.2
p.x_range.range_padding = 0.05

show(p)

Loading BokehJS ...